In [1]:
import numpy as np
import pandas as pd
import statsmodels.api as sm
from sqlalchemy import create_engine

import warnings
warnings.filterwarnings('ignore')

In [2]:
postgres_user = 'dsbc_student'
postgres_pw = '7*.8G9QH21'
postgres_host = '142.93.121.174'
postgres_port = '5432'
postgres_db = 'houseprices'

In [10]:
engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
    postgres_user, postgres_pw, postgres_host, postgres_port, postgres_db))
df = pd.read_sql_query('select * from houseprices',con=engine)

engine.dispose()

# Creating the model

In [11]:
numeric_variables = ['overallqual','firstflrsf','totrmsabvgrd']
variables = numeric_variables + ['neighborhood','exterior1st','saleprice']

In [12]:
df2 = pd.get_dummies(df[variables], drop_first=True)
X = df2.drop('saleprice', axis=1)
y = df2.saleprice

In [13]:
X = sm.add_constant(X)
results = sm.OLS(y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.787
Model:                            OLS   Adj. R-squared:                  0.781
Method:                 Least Squares   F-statistic:                     127.6
Date:                Sat, 24 Jul 2021   Prob (F-statistic):               0.00
Time:                        12:17:20   Log-Likelihood:                -17416.
No. Observations:                1460   AIC:                         3.492e+04
Df Residuals:                    1418   BIC:                         3.514e+04
Df Model:                          41                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -9.879e+04   1.46e+04     -6.776      0.000   -1.27e+05   -7.02e+04
overallqual           2.292e+04   1144.838     20.023      0.000    2.07e+04    2.52e+04
firstflrsf              43.1906      3.391     12.737      0.000      36.539      49.842
totrmsabvgrd          8869.1029    726.944     12.201      0.000    7443.102    1.03e+04
neighborhood_Blueste  1.755e+04   2.81e+04      0.624      0.533   -3.76e+04    7.27e+04
neighborhood_BrDale  -9569.3846   1.36e+04     -0.702      0.483   -3.63e+04    1.72e+04
neighborhood_BrkSide  1.507e+04   1.09e+04      1.388      0.165   -6225.160    3.64e+04
neighborhood_ClearCr  5.422e+04   1.19e+04      4.564      0.000    3.09e+04    7.75e+04
neighborhood_CollgCr  2.749e+04   9580.200      2.870      0.004    8701.504    4.63e+04
neighborhood_Crawfor  4.125e+04   1.09e+04      3.791      0.000    1.99e+04    6.26e+04
neighborhood_Edwards  5296.2822   1.03e+04      0.517      0.605   -1.48e+04    2.54e+04
neighborhood_Gilbert  2.818e+04   1.01e+04      2.777      0.006    8274.115    4.81e+04
neighborhood_IDOTRR  -2295.7572   1.16e+04     -0.199      0.843    -2.5e+04    2.04e+04
neighborhood_MeadowV -6450.1307   1.48e+04     -0.437      0.662   -3.54e+04    2.25e+04
neighborhood_Mitchel  1.592e+04   1.08e+04      1.480      0.139   -5185.857     3.7e+04
neighborhood_NAmes    1.128e+04   9811.629      1.150      0.250   -7962.302    3.05e+04
neighborhood_NPkVill  8126.8994   1.62e+04      0.502      0.615   -2.36e+04    3.99e+04
neighborhood_NWAmes   1.547e+04   1.04e+04      1.486      0.137   -4947.655    3.59e+04
neighborhood_NoRidge  1.062e+05   1.09e+04      9.707      0.000    8.47e+04    1.28e+05
neighborhood_NridgHt  7.771e+04   1.01e+04      7.717      0.000     5.8e+04    9.75e+04
neighborhood_OldTown  -853.6230   1.02e+04     -0.083      0.934   -2.09e+04    1.92e+04
neighborhood_SWISU     944.0728   1.22e+04      0.077      0.938    -2.3e+04    2.49e+04
neighborhood_Sawyer   1.459e+04   1.05e+04      1.390      0.165   -6002.141    3.52e+04
neighborhood_SawyerW   2.38e+04   1.05e+04      2.263      0.024    3165.265    4.44e+04
neighborhood_Somerst   3.87e+04      1e+04      3.858      0.000     1.9e+04    5.84e+04
neighborhood_StoneBr  8.127e+04   1.19e+04      6.807      0.000    5.79e+04    1.05e+05
neighborhood_Timber   4.282e+04   1.09e+04      3.912      0.000    2.13e+04    6.43e+04
neighborhood_Veenker  6.146e+04   1.48e+04      4.156      0.000    3.24e+04    9.05e+04
exterior1st_AsphShn  -6.394e+04   3.84e+04     -1.667      0.096   -1.39e+05    1.13e+04
exterior1st_BrkComm  -1.505e+04   2.79e+04     -0.540      0.589   -6.97e+04    3.96e+04
exterior1st_BrkFace   1.678e+04   1.01e+04      1.655      0.098   -3112.762    3.67e+04
exterior1st_CBlock    2.346e+04   3.82e+04      0.613    

# Evaluating the model

The F statistic is 127.6 which shows this model is better than an empty model.  The R-squared is 0.787 and the adjusted R-squared is 0.781 meaning 78 percent of the variance in saleprice can be explained by this model.  The AIC and BIC are both relatively large.  The model could use some improvement.  Once adjusted R-squared gets into the eighties while continuing to lower the AIC and BIC I will stop.

# Improving the model

From my simple linear regression model notebook, it appears saleprice may vary with the value of basement condition.  Since basement condition would not be correlated with the first floor exterior type, I will try to add this to the model and see if the model improves

In [14]:
variables2 = variables + ['bsmtcond']

In [15]:
df2 = pd.get_dummies(df[variables2], drop_first=True)
X = df2.drop('saleprice', axis=1)
y = df2.saleprice

In [16]:
X = sm.add_constant(X)
results = sm.OLS(y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.789
Model:                            OLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     120.1
Date:                Sat, 24 Jul 2021   Prob (F-statistic):               0.00
Time:                        12:17:31   Log-Likelihood:                -17409.
No. Observations:                1460   AIC:                         3.491e+04
Df Residuals:                    1415   BIC:                         3.515e+04
Df Model:                          44                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -1.089e+05   1.48e+04     -7.346      0.000   -1.38e+05   -7.98e+04
overallqual           2.204e+04   1170.291     18.829      0.000    1.97e+04    2.43e+04
firstflrsf              43.5062      3.382     12.864      0.000      36.872      50.140
totrmsabvgrd          9097.4610    727.409     12.507      0.000    7670.544    1.05e+04
neighborhood_Blueste  1.775e+04    2.8e+04      0.634      0.526   -3.72e+04    7.27e+04
neighborhood_BrDale  -8435.6162   1.36e+04     -0.620      0.535   -3.51e+04    1.82e+04
neighborhood_BrkSide  1.568e+04   1.08e+04      1.449      0.148   -5552.515    3.69e+04
neighborhood_ClearCr  5.456e+04   1.18e+04      4.608      0.000    3.13e+04    7.78e+04
neighborhood_CollgCr   2.75e+04   9549.953      2.880      0.004    8767.830    4.62e+04
neighborhood_Crawfor  4.138e+04   1.08e+04      3.816      0.000    2.01e+04    6.27e+04
neighborhood_Edwards  6309.1869   1.02e+04      0.617      0.537   -1.37e+04    2.64e+04
neighborhood_Gilbert  2.811e+04   1.01e+04      2.780      0.006    8275.719    4.79e+04
neighborhood_IDOTRR  -1135.0764   1.15e+04     -0.098      0.922   -2.38e+04    2.15e+04
neighborhood_MeadowV -7814.6436   1.47e+04     -0.531      0.595   -3.67e+04     2.1e+04
neighborhood_Mitchel  1.585e+04   1.07e+04      1.478      0.140   -5184.572    3.69e+04
neighborhood_NAmes    1.144e+04   9784.242      1.169      0.243   -7755.372    3.06e+04
neighborhood_NPkVill  8149.9981   1.61e+04      0.505      0.613   -2.35e+04    3.98e+04
neighborhood_NWAmes   1.532e+04   1.04e+04      1.477      0.140   -5028.643    3.57e+04
neighborhood_NoRidge  1.068e+05   1.09e+04      9.797      0.000    8.54e+04    1.28e+05
neighborhood_NridgHt  7.861e+04      1e+04      7.831      0.000    5.89e+04    9.83e+04
neighborhood_OldTown  -401.5360   1.02e+04     -0.039      0.969   -2.04e+04    1.96e+04
neighborhood_SWISU    2737.0455   1.22e+04      0.225      0.822   -2.11e+04    2.66e+04
neighborhood_Sawyer   1.402e+04   1.05e+04      1.339      0.181   -6516.378    3.46e+04
neighborhood_SawyerW  2.527e+04   1.05e+04      2.409      0.016    4690.660    4.59e+04
neighborhood_Somerst   3.92e+04   9994.848      3.922      0.000    1.96e+04    5.88e+04
neighborhood_StoneBr  8.209e+04   1.19e+04      6.899      0.000    5.87e+04    1.05e+05
neighborhood_Timber   4.344e+04   1.09e+04      3.982      0.000     2.2e+04    6.48e+04
neighborhood_Veenker  6.124e+04   1.47e+04      4.156      0.000    3.23e+04    9.01e+04
exterior1st_AsphShn  -5.277e+04   3.84e+04     -1.376      0.169   -1.28e+05    2.25e+04
exterior1st_BrkComm  -3795.7207    2.8e+04     -0.136      0.892   -5.87e+04    5.11e+04
exterior1st_BrkFace   1.554e+04   1.01e+04      1.537      0.125   -4292.491    3.54e+04
exterior1st_CBlock    1.939e+04   3.81e+04      0.509    

# Evaluating the new model

Both R-squared values and AIC values have improved, but not by much.  This model is better.  However the F statistic has decreased instead of increased and BIC has increased instead of decreased, supporting the first model.  Since the improvement in R-squared values and information criteria is not much I will try one more model, without basement condition.

# Attempting to improve the model more

For this model, I am going to include the variable that denotes wether or not the house has central air.

In [17]:
variables2 = variables + ['centralair']

In [18]:
df2 = pd.get_dummies(df[variables2], drop_first=True)
X = df2.drop('saleprice', axis=1)
y = df2.saleprice

In [19]:
X = sm.add_constant(X)
results = sm.OLS(y,X).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              saleprice   R-squared:                       0.788
Model:                            OLS   Adj. R-squared:                  0.782
Method:                 Least Squares   F-statistic:                     125.3
Date:                Sat, 24 Jul 2021   Prob (F-statistic):               0.00
Time:                        12:22:54   Log-Likelihood:                -17412.
No. Observations:                1460   AIC:                         3.491e+04
Df Residuals:                    1417   BIC:                         3.514e+04
Df Model:                          42                                         
Covariance Type:            nonrobust                                         
========================================================================================
                           coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------------
const                -1.033e+05   1.46e+04     -7.055      0.000   -1.32e+05   -7.45e+04
overallqual           2.238e+04   1159.450     19.299      0.000    2.01e+04    2.47e+04
firstflrsf              42.9745      3.384     12.699      0.000      36.336      49.613
totrmsabvgrd          8983.1416    726.453     12.366      0.000    7558.102    1.04e+04
neighborhood_Blueste   1.69e+04   2.81e+04      0.602      0.547   -3.81e+04    7.19e+04
neighborhood_BrDale  -1.043e+04   1.36e+04     -0.767      0.443   -3.71e+04    1.63e+04
neighborhood_BrkSide  1.574e+04   1.08e+04      1.453      0.146   -5511.970     3.7e+04
neighborhood_ClearCr  5.315e+04   1.19e+04      4.482      0.000    2.99e+04    7.64e+04
neighborhood_CollgCr  2.715e+04   9558.921      2.841      0.005    8402.953    4.59e+04
neighborhood_Crawfor  4.133e+04   1.09e+04      3.807      0.000       2e+04    6.26e+04
neighborhood_Edwards  5715.0559   1.02e+04      0.559      0.576   -1.43e+04    2.58e+04
neighborhood_Gilbert  2.781e+04   1.01e+04      2.747      0.006    7949.277    4.77e+04
neighborhood_IDOTRR    630.4715   1.16e+04      0.054      0.957   -2.21e+04    2.33e+04
neighborhood_MeadowV -8266.4622   1.47e+04     -0.561      0.575   -3.72e+04    2.06e+04
neighborhood_Mitchel   1.51e+04   1.07e+04      1.406      0.160   -5962.258    3.62e+04
neighborhood_NAmes    1.043e+04   9793.894      1.065      0.287   -8777.335    2.96e+04
neighborhood_NPkVill  7552.8015   1.61e+04      0.468      0.640   -2.41e+04    3.92e+04
neighborhood_NWAmes    1.49e+04   1.04e+04      1.435      0.151   -5468.161    3.53e+04
neighborhood_NoRidge  1.063e+05   1.09e+04      9.745      0.000    8.49e+04    1.28e+05
neighborhood_NridgHt  7.816e+04      1e+04      7.779      0.000    5.85e+04    9.79e+04
neighborhood_OldTown   282.4510   1.02e+04      0.028      0.978   -1.98e+04    2.03e+04
neighborhood_SWISU    2828.3476   1.22e+04      0.232      0.817   -2.11e+04    2.68e+04
neighborhood_Sawyer   1.368e+04   1.05e+04      1.305      0.192   -6878.546    3.42e+04
neighborhood_SawyerW  2.344e+04   1.05e+04      2.234      0.026    2855.777     4.4e+04
neighborhood_Somerst  3.869e+04      1e+04      3.867      0.000    1.91e+04    5.83e+04
neighborhood_StoneBr  8.172e+04   1.19e+04      6.859      0.000    5.84e+04    1.05e+05
neighborhood_Timber   4.267e+04   1.09e+04      3.908      0.000    2.12e+04    6.41e+04
neighborhood_Veenker   6.11e+04   1.48e+04      4.141      0.000    3.22e+04       9e+04
exterior1st_AsphShn  -5.772e+04   3.83e+04     -1.505      0.132   -1.33e+05    1.75e+04
exterior1st_BrkComm   -1.53e+04   2.78e+04     -0.550      0.582   -6.99e+04    3.93e+04
exterior1st_BrkFace   1.283e+04   1.02e+04      1.255      0.210   -7217.471    3.29e+04
exterior1st_CBlock    1.605e+04   3.83e+04      0.419    

# Evaluating the model

* F statistic -- still lower than the first
* R squared -- better than the first but not as good as the second
* Adjusted R squared --- Better than the first, the same as the second
* AIC -- Better than the first, the same as the second
* BIC -- The same as the first, better than the second.

# Choosing the model

Since my second and third attempt did not greatly improve the model evaluation metrics, I am going to choose my original model for house prices.